In [1]:
import random
import torch
import sys
from ImportantConfig import Config
config = Config()
from sql2fea import TreeBuilder,value_extractor
from NET import TreeNet
from sql2fea import Sql2Vec
from TreeLSTM import SPINN

sys.path.append('/home/ubuntu/project/HyperQO')

sys.stdout = open(config.log_file, "w")
random.seed(113)
with open(config.queries_file) as f:
    import json
    queries = json.load(f)

tree_builder = TreeBuilder()
sql2vec = Sql2Vec()
value_network = SPINN(head_num=config.head_num, input_size=37, hidden_size=config.hidden_size, table_num = 50,sql_size = 1).to(config.device)
for name, param in value_network.named_parameters():
    from torch.nn import init
    if len(param.shape)==2:
        init.xavier_normal(param)
    else:
        init.uniform(param)


treenet_model = TreeNet(tree_builder, value_network)

mask = (torch.rand(1,config.head_num,device = config.device)<0.9).long()

/tmp/ipykernel_17675/2163532279.py:25: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
  init.xavier_normal(param)
/tmp/ipykernel_17675/2163532279.py:27: UserWarning: nn.init.uniform is now deprecated in favor of nn.init.uniform_.
  init.uniform(param)


### train_dataloader
:sql,plan_json,target_value

In [2]:
import pandas as pd
train=pd.read_csv('./information/train.csv')
train.head()

,Unnamed: 0,query,cost_no_index,cost_dta,cost_reduction,cost_reduction_ratio
0,0,"select i_item_id, \n avg(ss_quantity) ...",108534.81,69925.31,38609.50,0.355734
1,1,"select sum (ss_quantity)\n from store_sales, s...",147137.68,86652.87,60484.81,0.411076
2,2,"select i_item_id\n ,i_item_desc \n ...",50155.33,3944.83,46210.50,0.921348
3,3,with ss_items as\n (select i_item_id item_id\n...,162566.89,839.12,161727.77,0.994838
4,4,"select asceding.rnk, i1.i_product_name best_p...",274060.77,48.66,274012.11,0.999822


In [12]:
x=train['query'].values
y=train['cost_reduction_ratio'].values

In [3]:
len(list(train['query'].values))

14400

In [4]:
from PGUtils import pgrunner
plan_json_PG_list=[]
for sql in train['query'].values:
    plan_json_PG_list.append(pgrunner.getCostPlanJson(sql))
# plan_json_PG_list[0]

In [5]:
train_dataloader=list(zip(train['query'].values, plan_json_PG_list, train['cost_reduction_ratio'].values))
# train_dataloader[0]

###  训练模型

In [6]:
num_epochs=1
# 训练模型
for epoch in range(num_epochs):
    total_loss = 0.0
    for batch_data in train_dataloader[:1000]:
        sql_vec,alias = sql2vec.to_vec(batch_data[0])
        plan_json, target_value = batch_data[1], batch_data[2]
        loss, mean, variance, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=True)
        total_loss +=loss
        display(loss)
    # 打印或记录训练过程中的损失等信息
    avg_loss = total_loss / 1000
    display(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

1.0958139896392822

0.8610769510269165


KeyboardInterrupt



In [ ]:
with torch.no_grad():
    for test_batch_data in train_dataloader[-9:]:
        sql_vec,alias = sql2vec.to_vec(test_batch_data[0])
        plan_json, target_value = test_batch_data[1], test_batch_data[2]
        _, mean, _, exp_variance = treenet_model.train(plan_json, sql_vec, target_value, mask, is_train=False)
        display(mean)

In [ ]:
[x[2] for x in train_dataloader[-9:]]

In [26]:
train_dataloader[:2]

[("select  i_item_id, \n        avg(ss_quantity) agg1,\n        avg(ss_list_price) agg2,\n        avg(ss_coupon_amt) agg3,\n        avg(ss_sales_price) agg4 \n from store_sales, customer_demographics, date_dim, item, promotion\n where ss_sold_date_sk = d_date_sk and\n       ss_item_sk = i_item_sk and\n       ss_cdemo_sk = cd_demo_sk and\n       ss_promo_sk = p_promo_sk and\n       cd_gender = 'F' and \n       cd_marital_status = 'M' and\n       cd_education_status = '2 yr Degree' and\n       (p_channel_email = 'N' or p_channel_event = 'N') and\n       d_year = 1999 \n group by i_item_id\n order by i_item_id\n limit 100;",
  {'Plan': {'Node Type': 'Limit',
    'Parallel Aware': False,
    'Startup Cost': 72519.65,
    'Total Cost': 72534.33,
    'Plan Rows': 100,
    'Plan Width': 145,
    'Plans': [{'Node Type': 'Aggregate',
      'Strategy': 'Sorted',
      'Partial Mode': 'Finalize',
      'Parent Relationship': 'Outer',
      'Parallel Aware': False,
      'Startup Cost': 72519.65,


In [24]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from pytorch_lightning import LightningModule, Trainer



# 定义 LightningModule 类
class MyLightningModule(LightningModule):
    def __init__(self, treenet_model, train_dataloader, mask):
        super().__init__()
        self.treenet_model = treenet_model
        self.train_dataloader = train_dataloader
        self.value_network = treenet_model.value_network
        self.mask = mask

    def forward(self, batch_data):
        sql_vec, alias = np.array([1]),set(['kt1'])
        plan_json, target_value = batch_data[1], batch_data[2]
        loss, mean, variance, exp_variance = self.treenet_model.train(plan_json, sql_vec, target_value, self.mask, is_train=False)
        return mean

    def training_step(self, batch, batch_idx):
        loss, mean, variance, exp_variance = self.treenet_model.train(plan_json, sql_vec, target_value, self.mask, is_train=False)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.value_network.parameters(), lr=3e-4)

# 创建 LightningModule 实例
model = MyLightningModule(treenet_model, train_dataloader, mask)

# 创建 Trainer 实例并训练模型
trainer = Trainer(max_epochs=1)  
trainer.fit(model, train_dataloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /home/ubuntu/project/HyperQO/lightning_logs

  | Name          | Type  | Params
----------------------------------------
0 | value_network | SPINN | 157 K 
----------------------------------------
157 K     Trainable params
0         Non-trainable params
157 K     Total params
0.630     Total estimated model params size (MB)
/home/ubuntu/anaconda3/envs/project/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


KeyboardInterrupt: 

In [22]:
import random

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # 示例数据
batch_size = 3  # 定义批次大小

# 首先打乱列表顺序
random.shuffle(data)

# 然后按照固定大小划分批次
num_batches = len(data) // batch_size
batches = [data[i * batch_size: (i + 1) * batch_size] for i in range(num_batches)]

# 输出随机划分的批次
for batch in batches:
    display(batch)


[6, 10, 5]

[2, 3, 8]

[9, 1, 7]